# Profile likelihood for $\Lambda$CDM

In [2]:
from lkl_prof_functions import *

### Read in chains 

- Set chains_dir as the folder where chain .txt files are found 
- Set chain_file as the filename up until the .1.txt etc
- Any chain settings you want, although these are irrelevant. We only use chains to check for bf file 
- Set minimizer settings here. Eg. if you're looking for a global minimum (besides testing, global minimum should be on). 
- Minimizer settings also inherits from the .yaml file for global minimum!! Ensure global minimum is also sought there 

In [3]:
chains_dir = "/Users/tanvikarwal/Desktop/Early_dark_energy/likelihood_profile/chains/lcdm_base/"
chain_file = 'lcdm_cmb_bao_sne_'
# chains_dir = '/home2/karwal/mcmc_chains/ede_lkl_profile/lcdm_base/'
os.chdir(chains_dir)

settings = {'ignore_rows' : 0.2}
mcmc_chains = mcsamples.loadMCSamples(chains_dir+chain_file, settings=settings)

minimizer_settings = {'minimize': 
                                {   'method': 'bobyqa',
                                    'covmat' : 'auto',
                                    # 'override_bobyqa' : {'seek_global_minimum': 'true'}
                                }
                      }
processes = 6

### Pick param to vary and increment

Ideally, only the parameter should be provided. 
- TBC: get increment from the MCMC sigma 
- TBC: get min and max from when $\chi^2$ has substantially dropped? 

For now, assuming these are input. 

(TBC = to be coded)

In [4]:
prof_param = 'H0'
prof_incr = 0.1
prof_min = 65.
prof_max = 74.

## Run profile likelihood code 

In [ ]:
param_order, MLs, lkl_profile = run_prof_lkl(chains_dir=chains_dir, chain_file=chain_file, mcmc_chains=mcmc_chains, processes=processes)

# Scrap code below

### Plot lkl profile
- Load proflkl param. Skip the first two rows because of proflkl output file error (bug now fixed)

In [15]:
param_names, param_ML, MLs = read_minimum(chains_dir, chain_file)
prof_param_values = np.loadtxt(chains_dir+chain_file+'_lkl_profile.txt', skiprows=2, usecols=(np.where(param_names == prof_param)[0][0],))

OSError: /Users/tanvikarwal/Desktop/Early_dark_energy/likelihood_profile/chains/lcdm_base/lcdm_cmb_bao_sne__lkl_prof.minimum not found.

- Load the $\chi^2$'s. Here, I'm doing each experiment separately because of a bug that messed up the proflkl output file. (bug now fixed) Note that this method might be more correct if you want to compare on $\chi^2$. The likelihood output by Cobaya includes normalisations and priors or something, such that $\left( -\log\mathcal{L} \right) \neq \frac{1}{2}\chi^2$
- Sum the $\chi^2$'s.

In [ ]:
params_wanted = ['chi2__CMB', 'chi2__SN', 'chi2__BAO']
chis = np.loadtxt(chains_dir+chain_file+'_lkl_profile.txt', skiprows=2, usecols=(np.where(param_names == i)[0][0] for i in params_wanted))
chis = np.array([ chis[i,0]+chis[i,1]+chis[i,2] for i in range(chis.shape[0]) ])

- Plot. That one should be obvs. 

In [ ]:
import matplotlib.pyplot as plt

plt.plot( prof_param_values, chis )
plt.ylabel(r'$\chi^2$')
plt.xlabel(r'$'+lkl_pro_yaml['params'][prof_param]['latex']+'$')
plt.savefig(chains_dir+chain_file+'_lkl_profile.png')

### Visualise how each parameter shifts with prof lkl param 

- Load all columns of output proflkl file
- plot all parameters in the file against the profile lkl parameter (this is the same as plotting against [1,2,3...]

In [ ]:
import matplotlib.pyplot as plt

all_MLs = np.loadtxt(chains_dir+chain_file+'_lkl_profile.txt', skiprows=2)
for i in range(all_MLs.shape[0]):
    plt.plot( all_MLs[:, np.where(param_names == prof_param)[0][0] ], all_MLs[:,i] )
    try:
        plt.ylabel(r'$'+lkl_pro_yaml['params'][param_names[i]]['latex']+'$')
    except KeyError:
        plt.ylabel([param_names[i]])
    plt.xlabel(r'$'+lkl_pro_yaml['params'][prof_param]['latex']+'$')
    plt.show()